In [19]:
import pandas as pd
import numpy as np
import random

import openai
import os
from tqdm import tqdm
import time

import sys

sys.path.insert(0, '..')

In [20]:
# TURBO COST
TURBO_COST = 0.00200 / 1000

In [21]:
# Initialize the API client
openai.api_key = os.environ.get("OPENAI_API_KEY")

In [98]:
df = pd.read_csv('../data/seed/youtube/seed_videos.csv')

In [128]:
df.head()

,Unnamed: 0,id,title,description,tags,raw_duration,duration,cleaned_transcript,raw_transcript,comments,comment_ids,label,link,snippet,messages
0,0,gFQNPmLKj1k,How to Invest for Beginners,"Hey friends, in this video I'll give you a com...","['Ali Abdaal', 'Ali abdal', 'how to get starte...",PT29M9S,00:29:09,"Right, so let's say\nyou want to get started w...","[{'text': ""Right, so let's say\nyou want to ge...","['Interesting ❤ Thank You ❤', 'Question about ...","['UgzTvJTisgQ45X-c3Ft4AaABAg', 'UgySEPzwDA4HNs...",Traditional,www.youtube.com/watch?v=gFQNPmLKj1k,How to Invest for Beginners. Maybe youve heard...,"[{'role': 'system', 'content': 'You are a clas..."
1,1,GZ42PIi9bis,I asked a personal finance expert how to invest.,Go to https://squarespace.com/mattdavella to s...,"[""Matt D'Avella"", 'matt davella', 'self-develo...",PT15M48S,00:15:48,"- Investing, when most people think of it, the...","[{'text': '- Investing, when most people think...",['Huge thanks to Ramit Sethi for joining me! S...,"['Ugx7vuOGQxrEMVYfVyN4AaABAg', 'Ugx7vuOGQxrEMV...",Traditional,www.youtube.com/watch?v=GZ42PIi9bis,I asked a personal finance expert how to inves...,"[{'role': 'system', 'content': 'You are a clas..."
2,2,Z3I8TPYn4Z0,Traditional and Roth IRAs | Simple Steps for a...,Individual Retirement Accounts (IRAs) can hel...,"['tdameritrade', 'TD Ameritrade', 'What is an ...",PT4M35S,00:04:35,"If you're saving for retirement, IRAs, also\nk...","[{'text': ""If you're saving for retirement, IR...",['hmmmmm do I think taxes will be HIGHER or LO...,"['UgxhIPwye-LL2ICHp594AaABAg', 'Ugw_poXl_sbpan...",Traditional,www.youtube.com/watch?v=Z3I8TPYn4Z0,Traditional and Roth IRAs | Simple Steps for a...,"[{'role': 'system', 'content': 'You are a clas..."
3,3,ss5yImerkhg,What is a 401(k)?,A 401(k) is an employer-sponsored retirement a...,"['401k', '401(k) contributions', '401k limit',...",PT5M23S,00:05:23,One of the most common ways people save for\nr...,[{'text': 'One of the most common ways people ...,['so if the market crashes what happens to you...,"['UgxvZb1p0tmadP_Lrn94AaABAg', 'UgxvZb1p0tmadP...",Traditional,www.youtube.com/watch?v=ss5yImerkhg,What is a 401(k)?. One of the most common ways...,"[{'role': 'system', 'content': 'You are a clas..."
4,4,DPnXJpDOAHI,How IRAs Work And Why They Are More Popular Th...,Choosing the right retirement account that bes...,"['CNBC', 'business', 'news', 'finance stock', ...",PT15M13S,00:15:13,are you one of the 60 million americans. acti...,[{'text': 'are you one of the 60 million ameri...,['I honestly hope they don’t pass laws that au...,"['UgyjWWVJM9zySO3L3oJ4AaABAg', 'Ugx-U2VntiMGdX...",Traditional,www.youtube.com/watch?v=DPnXJpDOAHI,How IRAs Work And Why They Are More Popular Th...,"[{'role': 'system', 'content': 'You are a clas..."


In [123]:
def create_messages(snippet):
    messages = [
        {"role": "system", "content" : "You are a classifier that determines if a YouTube video snippet falls under a label. A snippet is a concatenation of the video title, summarized transcript, and video tags. The labels and additional instructions will be included in the first user message."},
        {"role": "user", "content" : """Labels:

- Blockchain: Cryptocurrency, NFTs, or other Web3 investments
- Traditional: Stocks, Bonds, Real Estate, Commodities, or other traditional investments
- Mixed: Videos that recommend or educate about both blockchain and traditional investments
- None: Videos that do not recommend or educate about either blockchain or traditional investments

Instructions:

- If a video snippet mentions keywords related to blockchain or traditional investments but does not recommend or educate about the label, it will be classified as None.
- Please return predictions in the format" {Label} : {50 word or shorter rationale}"""},
        {"role": "assistant", "content": """Understood. I will classify YouTube video snippets based on the provided labels and instructions. Here's how I will format the predictions:

{Label} : {50-word or shorter rationale}

Please provide me with the YouTube video snippet you would like me to classify."""},
    ]
    
    snippet_message = {"role" : "user", "content" : snippet.replace("\n", " ").replace("  ", " ")}
    
    messages.append(snippet_message)
    
    return messages

In [109]:
def classify(messages):
    chatCompletion = openai.ChatCompletion.create(
      model="gpt-3.5-turbo",
      messages=messages,
      temperature=0.25
    )
    return chatCompletion

In [110]:
df['messages'] = df['snippet'].apply(create_messages)

In [111]:
completions = []
completion_df = []


pbar = tqdm(df.iterrows())  

for idx, entry in pbar:
    pbar.set_description("Processing %s" % entry['title'])
    
    body = classify(entry['messages'])
    
    ## Add error catching
    completion = dict(body['choices'][0]['message'])
    completion['prediction']  = completion['content'].split(':')[0].strip()
    completion['reason'] = completion['content'].split(':')[1].strip()
    
    ## Grab meta data
    completion.update({key: body[key] for key in ['created', 'id', 'model', 'object']})
    
    ## Grab token usage
    completion.update(dict(chatCompletion['usage']))

    completions.append(completion)
    
    pd.DataFrame(completions).to_csv('seed_predictions.csv', index_label = False)
    
    
    time.sleep(10)

Processing Investing Basics: Bonds: : 8it [00:38,  4.76s/it]                                                   


RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-YIcxNgcTtvdNQrKxaCXQIvbQ on requests per min. Limit: 20 / min. Current: 30 / min. Contact support@openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method.

In [112]:
predictions = pd.read_csv('seed_predictions.csv')

In [124]:
print(df['snippet'][2].replace("\n", " ").replace("  ", " "))

Traditional and Roth IRAs | Simple Steps for a Retirement Portfolio Course. A benefit of both traditional and Roth IRAs is that your earnings arent taxed while theyre in the account, which can help your investments compound. Where traditional and Roth IRAs differ is when you get tax breaks. With a traditional IRA, your contributions may be tax deductible. This means you may get a tax break in the years you contribute to the account. With a traditional IRA, you pay taxes on money you withdraw from the account during retirement. Essentially, choosing a traditional IRA means youll pay taxes in retirement, but you may get a tax deduction now. Once you contribute money to a Roth IRA, you wont have to pay taxes on your withdrawals during retirement. So choosing a Roth account means getting a tax break later. The biggest factor is whether you think your tax rate during retirement will be higher or lower than your tax rate during the years youre contributing. If you think your taxes are higher

In [ ]:
# Try using numerical labels as opposed to normal labels

In [119]:
for reason in predictions['reason']:
    print(reason + '\n')

The video snippet discusses investing in stocks, shares, and dividends, which are all traditional investments. The video does not mention blockchain or any other Web3 investments. The video also provides information on how much money is needed to get started and how risky buying shares in a company can be. Therefore, this video snippet falls under the Traditional label.

The video snippet focuses on educating viewers about traditional investments, specifically investing in the stock market through a 401k plan and a target retirement fund from Vanguard. The snippet does not mention blockchain or any other Web3 investments, and it does not recommend or educate about mixed investments. Therefore, it falls under the Traditional label.

The video snippet discusses both traditional and Roth IRAs, which are traditional investments, and explains the tax benefits of each. It also recommends diversifying tax savings by contributing to both types of accounts. However, it does not recommend or edu

In [127]:
sum(predictions['total_tokens']) * TURBO_COST

0.011392

In [114]:
predictions['prediction']

0    Traditional
1    Traditional
2          Mixed
3    Traditional
4          Mixed
5    Traditional
6    Traditional
7          Mixed
Name: prediction, dtype: object

In [102]:
df['label'].head(14)

0     Traditional
1     Traditional
2     Traditional
3     Traditional
4     Traditional
5     Traditional
6     Traditional
7     Traditional
8     Traditional
9     Traditional
10    Traditional
11    Traditional
12    Traditional
13    Traditional
Name: label, dtype: object